# Lab 3 Part II - Broadcast FM

Originally written by Miki Lustig.
<small>
    <p>Updated by Drake Lin in Spring 2023.</p>
    <p>Updated by Josh Sanz in Spring 2020, Spring 2022.</p>
    <p>Updated by Alan Dong in Spring 2021.</p>
</small>

In [1]:
import numpy as np, matplotlib.pyplot as plt
import matplotlib
from numpy import *
from numpy.fft import *
import scipy.signal as signal
from matplotlib.pyplot import *
from rtlsdr import RtlSdr
import scipy
import scipy.io
from scipy.io.wavfile import write
import sounddevice as sd
import gc
%matplotlib inline

Below are some functions from Part I that we will need in this part

In [2]:
# set the default card to be the built-in raspberry-pi audio
sd.default.device = 'bcm2835 Headphones: - (hw:0,0), ALSA'

def sg_plot(t_range, f_range, y, dbf=60, fig=None):
    """
    Plot an image of the spectrogram of y, with the axes labeled with time tl and frequency fl

    t_range -- time axis label, nt samples
    f_range -- frequency axis label, nf samples
    y -- spectrogram, (nf x nt) array
    dbf -- dynamic range of the spectrum in dB
    """
    eps = 10.0 ** ( -dbf / 20.0)  # minimum signal
    
    # find maximum
    y_max = abs(y).max()
    
    # compute 20*log magnitude, scaled to the max
    y_log = 20.0 * np.log10((abs(y) / y_max) * (1 - eps) + eps)
    
    # rescale image intensity to 256
    img = 256 * (y_log + dbf) / dbf - 1
    if fig is None:
        fig = plt.figure(figsize=(16, 6))
    plt.imshow(np.flipud(64.0 * (y_log + dbf) / dbf ), extent=t_range  + f_range, cmap=plt.cm.gray, aspect='auto')
    plt.xlabel('Time [s]')
    plt.ylabel('Frequency [Hz]')
    plt.tight_layout()

    return fig

## Task 5: Time-Frequency plots of the radio-frequency spectrum with the SDR.

The samples that are obtained by the SDR represent a bandwidth of the spectrum around a center frequency. 
Hence, when demodulating to baseband (i.e. zero frequency), the signal must be complex since it has an asymmetric Fourier Transform. 

In this case, we would like to display both sides of the spectrum.

* **Modify the function `myspectrogram_hann_ovlp(x,m,fs,fc)` such that it detects whether the input signal `x` is complex.**
* **In that case, it will compute a double sided spectrum with frequency range centered around fc (center frequency).** For this, it might be useful to use the commands: `np.isreal` and `fftshift`.


In [3]:
def myspectrogram_hann_ovlp(x, m, fs, fc, dbf = 60, fig = None):
    """
    Plot the spectrogram of x. If x is complex, plot the two-sided spectrum.
    
    First take the original signal x and split it into 50% overlapping blocks of length m.
    Then apply a Hann window to each block, FFT, and plot.
    """
    # pad x up to a multiple of m 
    lx = len(x)
    nt = (lx + m - 1) // m
    x = append(x,zeros(-lx+nt*m))
    
    # frequency index
    t_range = [0.0, lx / fs]
    
    # Your code here:

    # End of your code
    return fig

### Wideband FM Signal Background
We will first look at the spectrum of FM radio. In the US, the broadcast FM radio band is 88.0-108.0 MHz. It is split into 200 kHz wide slots. This is a relatively large bandwidth for FM signals, and therefore it is also called wideband FM as opposed to narrowband FM which can have a bandwidth as low as 5 KHz. In FM radio, the information is encoded by modulating the frequency of the carrier,  

$$y_c(t) = A\cos \left (2\pi f_c t + 2\pi \Delta f \int_0^t x(\tau) d\tau \right ).$$

Here, $f_c$ is the carrier frequency,  $\Delta f$ is a frequency deviation scalar and $x(t)$ is a normalized baseband signal, which contains all the information the station wants to broadcast. $\Delta f$ controls how wide the transmitted bandwidth is, since $x(t)$ is normalized.

The instantaneous frequency of a signal is the derivative of the phase. 
For the signal 

$$y_c(t) = A\cos \left (2\pi f_c t + 2\pi \Delta f \int_0^t x(\tau) d\tau \right ),$$ 

the phase is 

$$\phi(t) = 2\pi f_c t + 2\pi \Delta f \int_0^t x(\tau) d\tau,$$ 

and its instantaneous frequency is 

$$f(t) = \frac{1}{2\pi}\frac{d \phi}{dt} = f_c + \Delta f x(t)$$

which is exactly the signal we would like to transmit! 







The signal $x(t)$ is called the baseband signal. It has frequency content spanning from DC to about 100 KHz. This is in contrast to the modulated FM signal which has a bandwidth of 200 KHz (double sided spectrum) and is present at 88-108 MHz. For broadcast FM, the baseband signal includes stereo audio, as well as digital information about the station and sometimes additional narrow-bandwidth subcarrier channels.
More specifically, the broadcast FM baseband signal, $x(t)$, consists of a mono (Left+Right) channel from 30 Hz to 15 KHz, a pilot signal at $f_p=19$ KHz, an amplitude modulated stereo (Left-Right) channel around $2\cdot f_p = 38$ KHz and digital RBDS, which encodes the station information, song name, etc. at $3\cdot f_p = 57$ KHz. (See <http://en.wikipedia.org/wiki/FM_broadcasting> for more information). 

The baseband signal is:

$ \qquad \qquad x(t) = \underbrace{(L+R)}_{\mathrm{mono}} + \underbrace{0.1 \cdot \cos(2\pi f_p t)}_\mathrm{pilot} +  \underbrace{(L-R)\cos(2\pi (2f_p) t)}_\mathrm{stereo}  + \underbrace{0.05\cdot \mathrm{RBDS}(t)\cos(2\pi (3f_p) t)}_\mathrm{digital~ RBDS} + \mathrm{subcarrier~signals}. $

The signal $\mathrm{RBDS}(t)$ is $m(t)\cos(2\pi (3 f_p))$ where $m(t)$ is a binary signal consisting of $\pm1$ at constant intervals which encodes '0' and '1' bits. The subcarriers are often FM modulated signals of foreign stations or carry other information.
This is the spectrum of $x(t)$:

<center><img src="../imgs/FM.png" alt="FM" style="width: 800px;"/></center>
<center>Figure 1: Baseband broadcast FM spectrum</center>

We will listen to Berkeley's own KPFA 94.1 MHz station. KPFA's transmitter is located close to Grizzly Peak Road, close to where it meets Claremont Ave.

Recall that our SDR does IQ demodulation around a center frequency of choice, $f_d$, followed by a low-pass filter and sampling. This results in a complex digital signal: 

$$y_b(t) = Ae^{j2\pi (f_c-f_d) t + j2\pi \Delta f \int_0^t x(\tau) d\tau }.$$

When $f_d=f_c$ we get:

$$y_b(t) = Ae^{ j2\pi \Delta f \int_0^t x(\tau) d\tau }.$$

### System Implementation

We will implement the following system to demodulate and listen to KPFA:
<center><img src="../imgs/fmdemod_LPR.png" alt="FM" style="width: 800px;"/></center>
<center>Figure 2: Wideband FM demodulation system implementation</center>

We will provide details on each of these blocks below, but here's a brief overview:

- The SDR will provide samples at a BW of 960 KHz around the station frequency 94.1 MHz.
- The station BW is only 200 KHz, so we will create a low-pass filter to reject frequencies higher than $\pm 100$ KHz, and then decimate by 4 to reach a sampling rate of 240 KHz. 
- The limiter and the discriminator decode the frequency modulation to get our desired signal. The limiter gets rid of any undesired amplitude variation in the signal, and the discriminator converts frequency modulation into amplitude modulation. More later!
- After FM decoding, our desired signal occupies DC - 16 KHz. So we low-pass to reject all frequencies outside the band. Finally, we decimate by 5 to change to a sampling rate that can be played on the soundcard (48 KHz).

### Signal Capture:

Like always, you will get the best results if you collect samples outside. 


Task: 
* **Set the SDR to sample at a center frequency of 94.1 MHz (KPFA) and a rate of 960 KHz. Collect 480000 samples**, which is exactly half a second of data. We acquire only half a second because computing a spectrogram on more data may take a while on the Pi. 
- **Compute and display a spectrogram with a window of 1024 samples.** What is the spectral and temporal resolution? Explain what you see.

Tips:
- Don’t forget to play with different dynamic ranges of the spectrogram for best visualization. (40 worked well for us).
- Make sure to set the gain of the SDR so the signal is not under/overflowed! This will make a HUGE difference when demodulating.

In [5]:
fs = 960000
fc = 96.1e6
sdr = RtlSdr()
sdr.sample_rate = fs # sampling rate
sdr.center_freq = fc # center frequency
# Set the gain
# Your code here:

# End of your code
N_Samples = 480000
data = sdr.read_samples(N_Samples) # get samples
sdr.close()

In [6]:
# Plot. Plotting may take a while!
fig = #TODO


If you are having a hard time getting a good signal, your course staff collected 6 seconds of strong signal for you which you can use to test your code. 

* **When you submit, please submit with your own data collection if possible.**

One important difference between this data and the data you will collect is the SDR used has a 12-bit ADC and higher sampling rate (downsampled to match what you receive from the RTL-SDR), compared to 8-bits on your RTL-SDR. This means the ADC can represent up to 24 dB more dynamic range, and the process of downsampling can increase this value further. The limited dynamic range of your SDR means that the strong KPFA main broadcast can swamp the narrow and low power subcarrier signals. You may notice when listening to the audio later in the lab that when the KPFA broadcast is louder the subcarriers are noisier. Having a larger dynamic range means that the subcarriers are better represented in the provided recording and should be clearer when you listen to them.

In [4]:
data_full = np.load("../data/kpfa_12bit.npy") # 6 seconds of pre-recorded data with 12-bit ADC
# data = data_full[0:fs//2] # first 0.5 seconds

In [8]:
# Plot. Plotting may take a while!
fig = #TODO


## Task 6: Wideband FM Demodulation

### Downsampling:

The bandwidth of an FM station is 200 kHz. Since we sampled at 960 kHz, you may be seeing several stations in the spectrogram. KPFA, the station we are interested in is in the middle around 94.1 MHz.  We will need to filter out and select the desired station from the rest of the signals within the spectrum. 

* **Design an FIR low-pass filter.** 
* **The filter should have a length of 128 and a cutoff frequency of 100 kHz.** This will select the frequencies between -100 kHz to 100 kHz. 
* To design the filter, we will **use the function `signal.firwin`** (We will talk more about this filter design technique later in class -- but in essence the function creates a windowed sinc function with the appropriate bandwidth for the given length). 

Type `signal.firwin?` and execute to get help. 

    h = signal.firwin(128, 100000.0, nyq=fs/2, window='hanning')

* **Plot the double-sided log-scale magnitude frequency response of the filter** (log scale using `plt.semilogy`) by computing a zero-padded FFT to length 1024 and using fftshift. Use kHz as the x-axis units. 

In [9]:
h = signal.firwin(128,100000.0,nyq=fs/2,window='hanning')
H = #TODO
w = #TODO

# End of your code
fig=figure(figsize=(8,2))
plt.semilogy(w,abs(H))
plt.xlabel('Frequency [kHz]')
plt.title('Magnitude frequency response of the low-pass filter')

Now,
* **Filter the acquired SDR signal with the low-pass filter.** Use the function `numpy.convolve` or `signal.oaconvolve`. 
* **Plot the spectrogram of the filtered signal.**

In [147]:
# Your code here:



#### Decimation
In signal processing, the term "decimation" is used loosely to refer to either a simple reduction of the sampling rate by keeping every $M$'th sample (decimation by $M$) or the processing of low-pass filtering then sampling rate reduction to prevent aliasing. In this lab we will use the term downsampling for the full filtering and sample reduction process, and decimation to mean keeping every $M$'th sample.

As you would expect, if your signal bandwidth is the full Nyquist rate, $f_s/2$, then decimating without low-pass filtering would induce aliasing. However, we just filtered our signal to a bandwidth of 200 kHz from 960, so we can decimate up to a factor of almost 5 without aliasing.

- **Decimate the signal by a factor of 4** by selecting every 4th sample to get a signal sampled at a rate of 240 kHz. At this point, you have successfully implemented a downsampler through low-pass filtering and decimation! 
- **Plot the spectrogram with a window size of 512.** Remember to use the new sampling factor `fs/4.0` for the spectrogram.

Do you see the rolloff of the low-pass fiter? Is there aliasing?

In [148]:
# Your code here:



At this point you have gone through this processing with half a second of data. We will now acquire a longer segment, which will allow you to appreciate the audio that comes out once we demodulate it.

- **Acquire 960000*8 samples**, which is 8 seconds of data.
- **Filter and decimate (hence downsample) to the rate of 240 kHz.**
- **Plot the spectrogram _of the 1st second_**, i.e., of the first 240000 samples. 

In [5]:
fs = 960000
fc = 94.1e6
sdr = RtlSdr()
sdr.sample_rate = fs    # sampling rate
sdr.center_freq = fc   # center frequency
# Set the gain
# Your code here:

# End of your code

In [149]:
N_Samples = 960000 * 8
data = sdr.read_samples(N_Samples)   # get samples
sdr.close()

# save the data to load offline
np.save('../data/kpfa_12bit_mine.npy', data)

In [7]:
# use own data
data = np.load('../data/kpfa_12bit_mine.npy')
# use prerecorded data if needed
# data = data_full

In [8]:
# Filter and decimate using same filter from above
def filter_and_decimate(data):
    h = #TODO
    data_filtered = #TODO
    return data_filtered
        
data_f = filter_and_decimate(data)

In [9]:
# Plot spectrogram. This will take a while!
fig = #TODO


### FM Demodulation:

The plot does not resemble Fig. 1, the broadcast FM baseband, since it is not yet FM demodulated. We can easily see that the signal is frequency modulated -- because its frequency looks like a time-domain signal of speech or music.

The next step we are going to perform is to demodulate the FM signal and look at its spectrogram. For this we need to find the instantaneous frequency as a function of time, which is the time derivative of the phase. Directly computing the phase and then taking the derivative will be sensitive to phase wraps, which we would like to avoid.

Instead, we will use a digital implementation of classical FM demodulatation using a _limiter_ followed by a _discriminator_. The limiter makes the input have constant amplitude, and the discriminator converts frequency deviations into amplitude modulation. Just as a comment, implementation of an accurate analog disciminator is very difficult whereas implementing a digital one is quite easy!

#### Limiter
Recall that 

$$y_b(t) = A(t)e^{ j2\pi (f_c-f_d) t + j2\pi \Delta f \int_0^t x(\tau) d\tau }.$$ 

The leading coefficient $A(t)$ is some unwanted amplitude modulation, which can be a result of signal fading, noise, or other sources of signal variations. The role of the limiter component in an FM radio is to remove this amplitude modulation so the discriminator will only be sensitive to frequency variations. 

In the digital domain, implementing a limiter is done by simply dividing our signal elementwise by its amplitude. 

- **Apply a limiter to your signal.** To avoid dividing by 0, divide your signal by its amplitude plus $\epsilon$. $\epsilon = 10^{-6}$ is adequate. 




In [10]:
# Apply the limiter
def limiter(data_f):
    data_f = #TODO
    return data_f
data_f = limiter(data_f)

#### Discriminator
Assuming that $f_c=f_d$, after the limiter our signal is 

$$ y_b(t)=e^{ j2\pi \Delta f \int_0^t x(\tau) d\tau }.$$ 

To get $x(t)$ we can compute:

$$\left(\frac{d}{dt}y_b(t)\right)y_b^*(t) = j2\pi\Delta f \cdot x(t)\cdot e^{ j2\pi \Delta f \int_0^t x(\tau) d\tau }\cdot e^{  -j2\pi \Delta f \int_0^t x(\tau) d\tau } = j2\pi\Delta f \cdot x(t),$$

which gives us our desired signal when we take the imaginary part of the result. Notice that we mutltiply the differentiated signal $y_b'(t)$ by the complex conjugate of the original $y_b^*(t)$.

We will need to design a differentiator filter whose frequency response approximates the ideal frequency response, 

$$H_{diff}(e^{j\omega}) = \omega,$$

of a differentiator. Since we are only going to demodulate up to 105 kHz, we can allow the differentiator to roll off to zero after that.

- **Design an FIR differentiator filter with length 31 using the function `signal.remez`** which implements an equi-ripple min-max optimal filter design technique. It requires prescribing frequency bands and their corresponding frequency responses. We need to tell the function that it's a differentiator so it knows to match a linear frequency response within the band of interest. We will prescribe a linear frequency extending from 0-105 kHz and then tapering to zero at the Nyquist frequency 120 kHz. As we will learn later in class, since the filter is of even order (odd number of coefficients) and is antisymmetric, its Nyquist frequency must be zero!

Type `signal.remez?` for more information.

Specifically: 

    h_diff = signal.remez(31, [0.0,105000.0,120000.0,120000.0], [1.05/1.2,0], Hz=240000.0, type='differentiator')
    
will try to design a linear frequency response from 0-105000 Hz and zero amplitude at 120000 Hz.

- **Plot the filter and its two sided magnitude frequency response (linear scale, compute a zero-padded FFT to length 1024 and use fftshift).**


In [11]:
h_diff = signal.remez(31, [0.0,105000.0,120000.0,120000.0], [1.05/1.2,0], Hz=240000.0, type='differentiator')

# Compute the magnitude spectrum and plot both time and frequency domains.

H_diff = #TODO
w = #TODO

fig=figure(figsize=(16,2))
plt.stem(h_diff)
plt.xlabel('n')
plt.title('Impulse response of the differentiator filter')

fig=figure(figsize=(16,2))
plt.plot(w,abs(H_diff))
plt.xlabel('Frequency [kHz]')
plt.title('Magnitude frequency response of the differentiator filter')

Now, finish demoulating the FM signal:
- **Filter the signal with the differentiator**
- **Multiply the result with the conjugate of the original signal**
- **Take the imaginary component**

Note that the default implementation of `np.convolve` will have a delay of 16 samples with respect to the original signal. **To avoid that, use the option `mode='same'`.**

In [12]:
# Finish demodulating the signal
def freq_demod(data_f, h_diff):
    demod = #TODO
    return demod
demod = freq_demod(data_f, h_diff)

Note that after FM demodulation the signal should be real (by taking only the imaginary component) and hence only half the spectrum should be displayed.

- **Plot the spectrogram of the frequency demodulated signal and identify the mono audio, the pilot, the stereo and the RBDS signals.** Note that the RBDS signal may be too weak to detect or may need better spectral resolution.

In [13]:
# Plot the demodulated baseband signal
fig = #TODO


If you're using your own data, let's display the sampled data to **help identify the subcarriers**. 

In [14]:
data_full = np.load("../data/kpfa_12bit.npy")
demod_sample = freq_demod(limiter(filter_and_decimate(data_full)), h_diff)
fig = #TODO


### Play the mono  (Left  + Right Channel)

The mono signal covers the frequency range of 30 Hz - 16 kHz. However, there are many other signals present. There's another problem. Our sampled signal is at a rate of 240 kHz. The soundcard on most modern computers can only deal with a sampling rate of 48 kHz. Similar to the downsampling operation we did before, we must filter our signal and decimate it before we will be able to play it.

- **Design a 129-length FIR bandpass filter with a cutoff frequency of 16 kHz** using the command:

    h = signal.firwin(129,16000.0,nyq=240000.0/2,window='hanning')

- **Filter the signal and decimate by a factor of 5** to reduce the rate to 48 kHz. 
- **Store the result in the a variable called `LPR`** (Left + Right). 
- **Display the spectrogram of `LPR`.** Use a window length of 256. 

In [15]:
# Your code here:



#### Playing the Mono (Left + Right channel)

- **Normalize `LPR`** so it is in the range between -1 and 1. 
- **Play it.**

Do you hear radio?



In [16]:
# Your code here:



In [17]:
# Write to wav file
# write('../data/fm_LPR.wav', 48000, LPR)

## Task 7: Demodulating the subcarriers

There are two other signals in the spectrum, centered at 67.65 kHz and 92 kHz. These are subsidiary communications authorization (SCA) services. The FCC specifically doesn't regulate the FM band from 57 kHz up to 100 kHz, only requiring that whatever you transmit there doesn't interfere with the principle FM signal. For a long time, this extra spectrum was used for Muzak (a generic name for elevator music), and other targeted FM signals. This has become less common with the advent of the internet, but there are still stations that use these channels. KPFA transmits a Hatian-French subchannel at 67.65 kHz, and a Punjabi subchannel at 92 kHz.

Oddly enough, the subchannels themselves are FM encoded. This is like Russian Matryoshka dolls, with one inside another. Fortunately, you have all the tools you need to decode these signals! 

In order to listen to the subcarriers, first demodulate the subcarrier to baseband. Then low-pass to separate it from the other signals. Then, downsample it to 48 kHz, and finally frequency demodulate again using the technique described above.

The entire system is described in the following diagram:

<center><img src="../imgs/fmdemod_sc.png" alt="FM" style="width: 800px;"/></center>
<center>Figure 3: Subcarrier demodulation system implementation</center>

To demodulate the subcarrier to baseband, or zero frequency, we will need to:

* **Create a time index `t` representing the samples of the signal.**
* **Multiply the frequency demodulated signal in Task 6 with $e^{-j 2\pi f_0 t}$.**
* After shifting the frequency demodulated signal in Task 6 by $f_0=67.65$ kHz, **plot the spectrogram.** Since the signal is now complex, you should plot both sides of the spectrum. The subcarrier should be placed at the DC frequency.

Based on our experience, subcarrier 1 (Radio Papaya aka French Haitian) is down, but the Punjabi Radio is still broadcasting. Thus, if you want to demodulate both subcarriers use the recorded data, else feel free to use your own data, there will just be static for subcarrier 1.

In [18]:
# your data
data = np.load('../data/kpfa_12bit_mine.npy')

# recorded data
# data = np.load("../data/kpfa_12bit.npy")
# demod = freq_demod(limiter(filter_and_decimate(data_full)))

In [19]:
t = #TODO
subc1 = #TODO
fig = myspectrogram_hann_ovlp(subc1, 256, fs/4.0, 0, dbf=40)

To frequency demodulate, you will need to apply a limiter as before. The discriminator should be implemented in the same way as before. However, you need to design a new filter which approximates a differentiator between $\pm$8 kHz and then tapers to zero. Here's an example:

    h_diff = signal.remez(31, [0.0,8000.0,12000.0,24000], [8.0/24.0,0], Hz=48000, type='differentiator')
    
- **Design a 129-tap low-pass filter with a bandwidth of $\pm 6$ kHz in the passband.**
- **Filter the signal and decimate by a factor of 5** to get a signal with a sampling rate of 48 kHz. 
- **Plot the spectrogram** (make sure you adjust the spectrogram to represent the new sampling rate!). 
- **Perform the limiter operation.**
- **Apply the discriminator using the differentiator filter method.**
- **Low-pass the result with a 129-length FIR filter with cutoff of 5 kHz** to eliminate any residual high frequency noise. Don't forget that the sample rate is 48 kHz!
- **Scale the result to be within $\pm1$ and play the audio.**
- **Plot the spectrogram of the resulting signal.**

Can you hear French Hatian? 

In [20]:
def demod_subcarrier(subc):
    # this function decodes a subcarrier
    
    # filter and downsample by 5
    h = #TODO
    subc = #TODO
    fig = myspectrogram_hann_ovlp(subc1, 256, 48000, 0, dbf=20)

    h_diff = signal.remez(31, [0.0,8000.0,16000.0,24000], [8.0/24,0], Hz=48000, type='differentiator')
    subc = limiter(subc)
    subc_demod = freq_demod(subc, h_diff)

    # low pass filter
    h = #TODO
    subc_demod = #TODO

    fig = myspectrogram_hann_ovlp(subc_demod, 128, 48000, 0, dbf=30)

    return subc_demod/max(subc_demod)

# write('../data/fm_sc.wav', 48000, subc1_demod)

In [21]:
subc1_demod = demod_subcarrier(subc1)
sd.play(subc1_demod,48000,blocking=True)

Repeat the procedure to play the subcarrier (Punjabi channel) at $f_0=92$ kHz

In [22]:
t = #TODO
subc2 = #TODO
fig = myspectrogram_hann_ovlp(subc2, 1024, fs, 0, dbf=30)

subc2_demod = demod_subcarrier(subc2)
sd.play(subc2_demod,48000,blocking=True)

# write('fm_sc2.wav', 48000, subc2_demod)

In order to listen to the radio continuously, you would have to implement stream processing using a double buffer. The SDR would fill one buffer while your code demodulates the other one. Of course, you would need to process the data faster than the samples come in. Finally, you would also have to implement an overlapp and save/add approach for streaming convolutions so that the sound will not have interruptions or artifacts at the edges of the buffers.

Unfortunately, this is hard to implement using python on the Raspberry Pi because of inefficiencies in the way numpy uses memory. We will do more stream processing when we get to Lab 5.

## Task 8: Streaming FM audio

### CSDR
Fortunately, there are many people who are passionate about using low cost hardware to play with radio signals.  András Retzler (HA7ILM) created a website called [OpenWebRx](SDR.hu) which lets you view the spectrum at a receiver anywhere in the world over an internet connection. To help demodulate audio he wrote a lightweight but fast signal processing library called CSDR which works by connecting processing blocks sequentially via Unix pipes.

We will use CSDR to perform the same signal processing as above to demodulate an FM broadcast, but fast enough to sustain a (mostly) uninterrupted stream. You may wish to look at the readme on the [GitHub page](https://github.com/ha7ilm/csdr) to get an idea of how this works.

##### Aside: Running shell commands from Jupyter

It is possible to run shell (terminal / command line) commands from a Jupyter notebook with a Python kernel by putting "!" in front of the command. For example,
```
!ls
```
would run `ls` and display the result as the output of the cell.

If your shell command is taking too long, or runs forever until interrupted, you can send an interrupt to the process by clicking the "interrupt the kernel" button (black square) in the toolbar above.

In [ ]:
!ls

### Step 1: Acquire samples and decimate to 240 kHz

The support library for the RTL-SDR dongles that we are using includes a utility for streaming samples to a file or to `stdout` which allows us to stream data into CSDR. The command we will use is
```
rtl_sdr -s 960000 -f 94100000 -g 0 -n 960000 -
```
This command sets the sampling rate to 960 kHz, the center frequency to 94.1 MHz, and uses an automatic gain controller to set the gain based on the signal level. You can also set the gain parameter to whatever you found works best in your code above. The flag `-n 960000` tells the SDR to collect 960000 samples, or 1 second of data. This is useful for debugging your implementation. **For the final streaming radio playback you will want to use `-n 0` or no `-n` flag at all to tell the SDR to stream indefinitely.**

Once we have a stream of samples, we have to convert them from uint8_t to floats. We do this with 
```
csdr convert_u8_f
```

The next step in the process is to downsample to 240 kHz as above. CSDR has a command which applies a windowed low-pass filter then decimates called `fir_decimate_cc`. It takes three parameters defining the decimation rate, transition bandwidth (in the range 0-0.5), and window to apply (BLACKMAN, HAMMING, or BOX). 

You can downsample an IQ stream by a factor of, for example, 10 by piping the `rtl_sdr` process together with the downsampler:
```
rtl_sdr -s 960000 -f 94100000 -g 0 -n 960000- | csdr convert_u8_f | csdr fir_decimate_cc 10 .05 HAMMING
```

Remember that we want to downsample by 4. You can test whether you are using the correct settings by writing the output of the downsampler to a file, then reading the file and running the rest of your demodulation code on the contents of the file.

* **Write one second of downsampled data to a file called `downsample.bin`.**
* **Downsample by 4 with 0.05 transition bandwidth and a Hamming window.**

In [50]:
# Write to file
# !rtl_sdr -s 960000 -f 94100000 -g 0 -n 960000 - | csdr convert_u8_f | csdr fir_decimate_cc <rate> <f_cutoff> <window> > downsample.bin
# Your code here:



In [32]:
# Plot the spectrogram of the recording (first second)
# CSDR sets the cutoff frequency to exactly the downsampled Nyquist frequency, 
# so there will be at most a slight reduction in energy near the edges.
d_csdr = np.fromfile("downsample.bin", dtype=np.complex64)
fig = myspectrogram_hann_ovlp(d_csdr, 512, 240000, fc, dbf=40)

### Step 2: FM demodulate
After downsampling, we again want to demodulate. CSDR accomplishes this with a block called `fmdemod_quadri_cf`. This blocks uses a related, but slightly simpler method for demodulation. In order to extract the instantaneous phase of the signal, the demodulator calculates the phase of each sample then uses the previous sample to approximate the rate of change of phase.

Look at the [source](https://github.com/ha7ilm/csdr/blob/master/libcsdr.c) line 1024 for the implementation. [https://www.embedded.com/dsp-tricks-frequency-demodulation-algorithms/] has a derivation of how $\frac{d}{dt}\left(\tan^{-1}\left(\frac{Q(t)}{I(t)}\right)\right)$ becomes the code that you see.

When comparing to the differentation (discriminator) you implemented above, notice how your discriminator uses 31 taps but the CSDR code only uses 2. Using fewer taps makes implementing the algorithm easier and more efficient, but the quality of the output is degraded more by noise, since there is no smoothing across multiple samples. This is a tradeoff that occurs over and over again in real systems - implementation ease and efficiency versus performance of the processing algorithm.

* **Store the output of the demodulator to `demodulate.bin` and plot the spectrogram.**

In [52]:
# Your beautiful command
# !TODO > demodulate.bin
# Your code here:



In [31]:
# Plot the spectrogram of the recording (first second)
# You should see the audio and the 19 kHz pilot tones, and possibly the subcarriers.
d_csdr = np.fromfile("demodulate.bin", dtype=np.float32)
fig = myspectrogram_hann_ovlp(d_csdr, 512, 240000, 0, dbf=40)

Now we again downsample to the mono audio stream to reach 48 kHz for the audio card. This time, because the sample stream is real, we use the `fractional_decimator_ff` command which takes a single argument, the decimation (downsampling) rate. 

* **Downsample by a factor of 5 using `fractional_decimator_ff`, save to `audio_f.bin`, and plot the spectrogram.**

In [113]:
# Your beautiful command
# !TODO > audio_f.bin
# Your code here:



In [30]:
# Plot the spectrogram of the recording (first second)
d_csdr = np.fromfile("audio_f.bin", dtype=np.float32)
fig = myspectrogram_hann_ovlp(d_csdr, 512, 48000, 0, dbf=40)

### Step 3: Play the audio
Finally, we output the stream of samples as signed 16-bit ints to the soundcard.
```
!<your demodulation commands> | csdr convert_f_s16 | aplay -t raw -r 48000 -f s16_le -c 1 -D hw:0,0
```
Don't forget to change the n-samples flag to infinite: `-n 0`.

In [29]:
# Your audio streaming command
# Your code here:

